In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings(action="ignore")

Load the houseprices data from Thinkful's database.

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices',con=engine)
engine.dispose()

In [3]:
## Fill continuous variable null values with zero
for column in ['masvnrarea', 'lotfrontage', 'garagecars']:
    df[column] = df[column].fillna(0)
    
indexes = df[df.garageyrblt == 'None'].index
df.loc[indexes, ['garageyrblt']] = 1980
df['garageyrblt'] = pd.to_numeric(df['garageyrblt'])

## Fill all null values with 'none'
df = df.fillna('None')

In [7]:
df['ttl_sq_ft'] = df.totalbsmtsf + df.firstflrsf + df.secondflrsf
df['sqfXqual'] = df.ttl_sq_ft * df.overallqual
df = pd.concat([df, pd.get_dummies(df.mszoning, drop_first=True)], axis=1)

df2 = df[['yearbuilt', 'yearremodadd', 'bsmtfinsf1', 'fireplaces', 'garagecars',
          'wooddecksf', 'secondflrsf', 'FV', 'RH', 'RL', 'RM',
          'saleprice', 'ttl_sq_ft', 'sqfXqual', 'masvnrarea']]

Split your data into train and test sets.

In [15]:
Y = df2.saleprice
X = df2.drop(columns = ['saleprice'])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .02, random_state = 390)

Estimate your model from the previous checkpoint in the train set. Assess the goodness of fit of your model.

In [16]:
X_train = sm.add_constant(X_train)
lrm = sm.OLS(Y_train, X_train).fit()
lrm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.793
Method:                 Least Squares   F-statistic:                     422.7
Date:                Sun, 18 Aug 2019   Prob (F-statistic):               0.00
Time:                        17:30:57   Log-Likelihood:                -17030.
No. Observations:                1430   AIC:                         3.409e+04
Df Residuals:                    1416   BIC:                         3.416e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -8.911e+05    1.2e+05     -7.452      0.000   -1.13e+06   -6.57e+05
yearbuilt       70.4342     48.309      1.458      0.145     -24.330     165.198
yearremodadd   398.6370     60.812      6.555      0.000     279.345     517.929
bsmtfinsf1      11.4324      2.481      4.609      0.000       6.566      16.299
fireplaces    1.002e+04   1736.375      5.768      0.000    6609.568    1.34e+04
garagecars    1.653e+04   1723.117      9.594      0.000    1.32e+04    1.99e+04
wooddecksf      30.7289      8.200      3.747      0.000      14.643      46.815
secondflrsf     14.3229      2.524      5.675      0.000       9.372      19.274
FV            2.674e+04   1.32e+04      2.021      0.043     784.780    5.27e+04
RH            2.121e+04   1.51e+04      1.403      0.161   -8450.376    5.09e+04
RL            2.627e+04   1.23e+04      2.141      0.032    2199.538    5.03e+04
RM            1.032e+04   1.23e+04      0.836      0.403   -1.39e+04    3.45e+04
ttl_sq_ft      -16.0788      3.654     -4.400      0.000     -23.247      -8.910
sqfXqual         6.7719      0.364     18.599      0.000       6.058       7.486
==============================================================================
Omnibus:                     1520.240   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           606122.471
Skew:                          -4.462   Prob(JB):                         0.00
Kurtosis:                     103.464   Cond. No.                     2.37e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.37e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

As discussed in the last checkpoint, the r-squared is decently high, and the AIC and BIC are low

Predict the house prices in the test set, and evaluate the performance of your model using the metrics we mentioned in this checkpoint.

In [17]:
X_test = sm.add_constant(X_test)
Y_pred = lrm.predict(X_test)

print('MAE: {}'.format(mean_absolute_error(Y_test, Y_pred)))
print('MSE: {}'.format(mse(Y_test, Y_pred)))
print('RMSE: {}'.format(rmse(Y_test, Y_pred)))
print('MAPE: {}'.format(((Y_test - Y_pred) / Y_test).abs().mean()))

MAE: 14406.753189022818
MSE: 365858659.1114245
RMSE: 19127.432109706322
MAPE: 0.09309841449249294


Is the performance of your model satisfactory? Why?

MAPE looks impressively low. I would say that makes the model satisfactory.

Try to improve your model in terms of predictive performance by adding or removing some variables.

In [19]:
Y = np.log1p(df2.saleprice)
X = df2.drop(columns = ['saleprice'])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .02, random_state = 390)

In [20]:
X_train = sm.add_constant(X_train)
lrm = sm.OLS(Y_train, X_train).fit()
lrm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.825
Model:                            OLS   Adj. R-squared:                  0.824
Method:                 Least Squares   F-statistic:                     515.1
Date:                Sun, 18 Aug 2019   Prob (F-statistic):               0.00
Time:                        17:32:32   Log-Likelihood:                 535.89
No. Observations:                1430   AIC:                            -1044.
Df Residuals:                    1416   BIC:                            -970.1
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            2.1523      0.553      3.891      0.000       1.067       3.237
yearbuilt        0.0011      0.000      4.917      0.000       0.001       0.002
yearremodadd     0.0033      0.000     11.790      0.000       0.003       0.004
bsmtfinsf1    4.205e-05   1.15e-05      3.664      0.000    1.95e-05    6.46e-05
fireplaces       0.0758      0.008      9.437      0.000       0.060       0.092
garagecars       0.1080      0.008     13.553      0.000       0.092       0.124
wooddecksf       0.0001   3.79e-05      3.616      0.000    6.27e-05       0.000
secondflrsf    8.13e-05   1.17e-05      6.964      0.000    5.84e-05       0.000
FV               0.4880      0.061      7.973      0.000       0.368       0.608
RH               0.4125      0.070      5.897      0.000       0.275       0.550
RL               0.4523      0.057      7.968      0.000       0.341       0.564
RM               0.3275      0.057      5.733      0.000       0.215       0.440
ttl_sq_ft     5.638e-05   1.69e-05      3.335      0.001    2.32e-05    8.95e-05
sqfXqual      1.518e-05   1.68e-06      9.016      0.000    1.19e-05    1.85e-05
==============================================================================
Omnibus:                     1377.783   Durbin-Watson:                   2.044
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           195705.001
Skew:                          -4.065   Prob(JB):                         0.00
Kurtosis:                      59.732   Cond. No.                     2.37e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.37e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
X_test = sm.add_constant(X_test)
Y_pred = lrm.predict(X_test)

print('MAE: {}'.format(mean_absolute_error(Y_test, Y_pred)))
print('MSE: {}'.format(mse(Y_test, Y_pred)))
print('RMSE: {}'.format(rmse(Y_test, Y_pred)))
print('MAPE: {}'.format(((Y_test - Y_pred) / Y_test).abs().mean()))

MAE: 0.09267718641441244
MSE: 0.018675924992714418
RMSE: 0.13665988801661744
MAPE: 0.007897929205483064
